# 🚀 Galactica to Diffusion Model - 完整训练版

**环境**: AutoDL GPU实例  
**GPU**: RTX 5090 (32GB)  
**论文**: [Scaling Diffusion Language Models via Adaptation from Autoregressive Models](https://arxiv.org/abs/2410.17891)
**数据集**: ArXiv论文数据集

---

## 📋 Notebook结构

1. 环境检查和配置
2. 导入库和定义配置类
3. 核心Diffusion函数
4. 数据集类
5. 模型包装类
6. 模型加载
7. **数据准备（ArXiv数据集）**
8. 训练循环（包含数值稳定性检查）
9. 训练可视化
10. 生成测试

---

## 1️⃣ 环境检查和配置

In [1]:
# 忽略警告
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

import torch

print("="*60)
print("环境检查")
print("="*60)

print(f"\n✓ PyTorch版本: {torch.__version__}")
print(f"✓ CUDA可用: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ 显存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"✓ CUDA版本: {torch.version.cuda}")
else:
    print("⚠️  警告: GPU不可用，将使用CPU（训练会很慢）")

print("\n" + "="*60)
print("✓ 环境检查完成")
print("="*60)

环境检查

✓ PyTorch版本: 2.7.0+cu128
✓ CUDA可用: True
✓ GPU: NVIDIA GeForce RTX 5090
✓ 显存: 33.7 GB
✓ CUDA版本: 12.8

✓ 环境检查完成


## 2️⃣ 导入库和定义配置类

In [2]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, get_linear_schedule_with_warmup
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from dataclasses import dataclass
import time
import json

print("✓ 所有库导入成功")

✓ 所有库导入成功


In [3]:
# 配置类
@dataclass
class TrainingConfig:
    """训练配置参数"""
    
    # 模型路径
    model_path: str = "/root/.cache/modelscope/hub/models/facebook/galactica-125m"
    
    # 数据路径
    data_path: str = "./autodl-tmp/arxiv_train-1.jsonl"  # ArXiv处理后的数据
    
    # 训练参数
    batch_size: int = 8  # 降低batch size提高稳定性
    gradient_accumulation_steps: int = 4  # 梯度累积
    num_epochs: int = 3
    learning_rate: float = 1e-4  # 降低学习率
    warmup_steps: int = 500
    max_grad_norm: float = 0.5  # 严格的梯度裁剪
    
    # 数据参数
    max_length: int = 128
    max_train_samples: int = None  # None表示使用全部数据，可设置为10000进行快速测试
    
    # Diffusion参数
    mask_token_id: int = None  # 将在加载tokenizer后自动设置
    
    # 设备和输出
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    output_dir: str = "/root/autodl-tmp/diffusion_galactica_checkpoints"
    save_steps: int = 1000
    
    # 数值稳定性
    use_fp32: bool = True  # 使用FP32提高数值稳定性
    check_nan: bool = True  # 启用NaN检测

config = TrainingConfig()
print("✓ 配置已加载")
print(f"\n配置摘要:")
print(f"  - 设备: {config.device}")
print(f"  - 数据路径: {config.data_path}")
print(f"  - Batch size: {config.batch_size}")
print(f"  - 学习率: {config.learning_rate}")
print(f"  - 最大序列长度: {config.max_length}")
print(f"  - 最大训练样本: {config.max_train_samples if config.max_train_samples else '全部'}")
print(f"  - 使用FP32: {config.use_fp32}")

✓ 配置已加载

配置摘要:
  - 设备: cuda
  - 数据路径: ./autodl-tmp/arxiv_train-1.jsonl
  - Batch size: 8
  - 学习率: 0.0001
  - 最大序列长度: 128
  - 最大训练样本: 全部
  - 使用FP32: True


## 3️⃣ 核心Diffusion函数

In [4]:
def forward_diffusion_step(x0, t, mask_token_id, device="cuda"):
    """
    前向diffusion过程：根据时间步t将x0转换为xt
    
    Args:
        x0: 原始token序列 [batch_size, seq_len]
        t: 时间步 [batch_size] (范围0-1)
        mask_token_id: mask token的ID
    
    Returns:
        xt: 加噪后的序列 [batch_size, seq_len]
        mask: 哪些位置被mask了 [batch_size, seq_len]
    """
    batch_size, seq_len = x0.shape
    
    # 为每个样本生成独立的mask概率
    mask_prob = t.unsqueeze(1).expand(-1, seq_len)
    
    # 生成随机mask
    random_values = torch.rand(batch_size, seq_len, device=device)
    mask = random_values < mask_prob
    
    # 应用mask
    xt = x0.clone()
    xt[mask] = mask_token_id
    
    return xt, mask


def compute_diffusion_loss(logits, x0, mask, t):
    """
    计算diffusion损失（带数值稳定性检查）
    
    Args:
        logits: 模型输出 [batch_size, seq_len, vocab_size]
        x0: 真实token [batch_size, seq_len]
        mask: mask位置 [batch_size, seq_len]
        t: 时间步 [batch_size]
    """
    # 检查输入数值稳定性
    if torch.isnan(logits).any():
        print("⚠️  警告: logits包含NaN")
        logits = torch.nan_to_num(logits, nan=0.0)
    
    if torch.isinf(logits).any():
        print("⚠️  警告: logits包含Inf")
        logits = torch.nan_to_num(logits, posinf=1e4, neginf=-1e4)
    
    batch_size, seq_len, vocab_size = logits.shape
    
    # 只计算被mask位置的损失
    logits_flat = logits.reshape(-1, vocab_size)
    targets_flat = x0.reshape(-1)
    mask_flat = mask.reshape(-1)
    
    # 计算交叉熵损失（不进行reduction）
    ce_loss = F.cross_entropy(logits_flat, targets_flat, reduction='none')
    
    # 只保留mask位置的损失
    ce_loss = ce_loss * mask_flat.float()
    
    # 检查损失的数值稳定性
    if torch.isnan(ce_loss).any() or torch.isinf(ce_loss).any():
        print("⚠️  警告: 损失包含NaN或Inf，跳过此批次")
        return None
    
    # 时间步加权（较小的t权重更大）
    if mask_flat.sum() > 0:
        t_expanded = t.unsqueeze(1).expand(-1, seq_len).reshape(-1)
        t_for_masked = t_expanded[mask_flat]
        
        # 避免除以0
        weights = 1.0 / (t_for_masked + 1e-8)
        weighted_loss = ce_loss[mask_flat] * weights
        
        return weighted_loss.mean()
    else:
        return ce_loss.mean()


print("✓ Diffusion核心函数定义完成")

✓ Diffusion核心函数定义完成


## 4️⃣ 数据集类

In [5]:
class DiffusionDataset(Dataset):
    """Diffusion训练数据集"""
    
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        
        # Tokenize
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0)
        }

print("✓ DiffusionDataset类定义完成")

✓ DiffusionDataset类定义完成


## 5️⃣ 模型包装类

In [6]:
class DiffusionGalactica(nn.Module):
    """包装Galactica模型以支持diffusion训练"""
    
    def __init__(self, base_model):
        super().__init__()
        self.model = base_model
        self.config = base_model.config
    
    def forward(self, input_ids, attention_mask=None):
        """
        前向传播
        
        Args:
            input_ids: token IDs [batch_size, seq_len]
            attention_mask: attention mask [batch_size, seq_len]
        
        Returns:
            logits: [batch_size, seq_len, vocab_size]
        """
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        
        return outputs.logits

print("✓ DiffusionGalactica类定义完成")

✓ DiffusionGalactica类定义完成


## 6️⃣ 模型加载

In [7]:
print("="*60)
print("加载模型")
print("="*60)

# 创建输出目录
os.makedirs(config.output_dir, exist_ok=True)

print(f"\n1. 加载Galactica-125M模型...")
print(f"   路径: {config.model_path}")

# 检查模型路径
if not os.path.exists(config.model_path):
    raise FileNotFoundError(
        f"模型路径不存在: {config.model_path}\n"
        f"请确保已通过ModelScope下载模型"
    )

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.model_path)

# 设置padding token（关键修复）
if tokenizer.pad_token is None:
    if tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"   ✓ 设置pad_token为eos_token: {tokenizer.eos_token}")
    else:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        print("   ✓ 添加新的padding token: [PAD]")

print(f"   Pad token: {tokenizer.pad_token}")
print(f"   Pad token ID: {tokenizer.pad_token_id}")

# ✅ 设置mask_token_id（修复CUDA索引错误）
config.mask_token_id = tokenizer.pad_token_id
print(f"   Mask token ID: {config.mask_token_id}")
print(f"   词表大小: {len(tokenizer)}")

# 验证mask_token_id在有效范围内
assert config.mask_token_id is not None, "mask_token_id未设置"
assert config.mask_token_id < len(tokenizer), \
    f"mask_token_id ({config.mask_token_id}) 超出词表大小 ({len(tokenizer)})"
print("   ✓ mask_token_id验证通过")

# 加载基础模型
dtype = torch.float32 if config.use_fp32 else torch.float16
base_model = AutoModelForCausalLM.from_pretrained(
    config.model_path,
    torch_dtype=dtype,
    device_map=None  # 手动管理设备
)

# 如果添加了新的pad token，调整词表大小
if tokenizer.pad_token == '[PAD]':
    base_model.resize_token_embeddings(len(tokenizer))
    print("   ✓ 调整模型词表大小以适应新的pad token")

# 包装为diffusion模型并移到设备
model = DiffusionGalactica(base_model)
model = model.to(config.device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✓ 模型加载完成")
print(f"   总参数: {total_params:,}")
print(f"   可训练参数: {trainable_params:,}")
print(f"   设备: {next(model.parameters()).device}")
print(f"   数据类型: {next(model.parameters()).dtype}")
print("="*60)

加载模型

1. 加载Galactica-125M模型...
   路径: /root/.cache/modelscope/hub/models/facebook/galactica-125m
   ✓ 添加新的padding token: [PAD]
   Pad token: [PAD]
   Pad token ID: 50000
   Mask token ID: 50000
   词表大小: 50001
   ✓ mask_token_id验证通过
   ✓ 调整模型词表大小以适应新的pad token

✓ 模型加载完成
   总参数: 125,031,168
   可训练参数: 125,031,168
   设备: cuda:0
   数据类型: torch.float32


## 7️⃣ 数据准备（ArXiv数据集）

In [8]:
print("="*60)
print("数据准备")
print("="*60)

def load_training_data(data_path, max_samples=None):
    """
    加载处理好的训练数据
    
    Args:
        data_path: 数据文件路径 (.txt 或 .jsonl)
        max_samples: 最大样本数（None表示加载全部）
    
    Returns:
        texts: 文本列表
    """
    from pathlib import Path
    
    data_path = Path(data_path)
    
    if not data_path.exists():
        raise FileNotFoundError(
            f"数据文件不存在: {data_path}\n"
            f"请先运行 process_arxiv_dataset.py 处理数据"
        )
    
    texts = []
    
    # 根据文件扩展名选择读取方式
    if data_path.suffix == '.txt':
        print(f"从文本文件加载: {data_path}")
        with open(data_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if max_samples and i >= max_samples:
                    break
                text = line.strip()
                if text:  # 跳过空行
                    texts.append(text)
    
    elif data_path.suffix == '.jsonl':
        print(f"从JSONL文件加载: {data_path}")
        with open(data_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if max_samples and i >= max_samples:
                    break
                data = json.loads(line)
                texts.append(data['text'])
    
    else:
        raise ValueError(f"不支持的文件格式: {data_path.suffix}，请使用 .txt 或 .jsonl")
    
    return texts


# 加载训练数据
print("\n1. 加载训练数据...")
try:
    train_texts = load_training_data(
        data_path=config.data_path,
        max_samples=config.max_train_samples
    )
    
    print(f"\n✓ 加载完成: {len(train_texts)} 条训练样本")
    
    # 打印统计信息
    avg_len = sum(len(t) for t in train_texts) / len(train_texts) if train_texts else 0
    max_len = max(len(t) for t in train_texts) if train_texts else 0
    min_len = min(len(t) for t in train_texts) if train_texts else 0
    
    print(f"\n数据统计:")
    print(f"  样本数: {len(train_texts)}")
    print(f"  平均长度: {avg_len:.0f} 字符")
    print(f"  最大长度: {max_len} 字符")
    print(f"  最小长度: {min_len} 字符")
    
    # 显示第一个样本的预览
    if train_texts:
        print(f"\n第一个样本预览 (前200字符):")
        print("-" * 60)
        print(train_texts[0][:200] + "...")
        print("-" * 60)
    
except FileNotFoundError as e:
    print(f"\n❌ 错误: {e}")
    print("\n请按照以下步骤准备数据:")
    print("  1. 确保已下载arxiv数据集的parquet文件")
    print("  2. 运行: python process_arxiv_dataset.py /path/to/parquet/files")
    print("  3. 确保生成了 ./processed_data/arxiv_train.txt 文件")
    raise

# 创建数据集
print("\n2. 创建数据集...")
train_dataset = DiffusionDataset(
    texts=train_texts,
    tokenizer=tokenizer,
    max_length=config.max_length
)

# 创建DataLoader
print("3. 创建DataLoader...")
train_dataloader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True if config.device == "cuda" else False
)

print(f"\n✓ 数据准备完成")
print(f"  训练样本数: {len(train_dataset)}")
print(f"  Batch大小: {config.batch_size}")
print(f"  Batch数量: {len(train_dataloader)}")
print(f"  每个epoch步数: {len(train_dataloader)}")
print(f"  使用梯度累积: {config.gradient_accumulation_steps} 步")
print(f"  有效batch size: {config.batch_size * config.gradient_accumulation_steps}")
print("="*60)

数据准备

1. 加载训练数据...
从JSONL文件加载: autodl-tmp/arxiv_train-1.jsonl

✓ 加载完成: 32094 条训练样本

数据统计:
  样本数: 32094
  平均长度: 33456 字符
  最大长度: 657995 字符
  最小长度: 101 字符

第一个样本预览 (前200字符):
------------------------------------------------------------
additive models @xcite provide an important family of models for semiparametric regression or classification . some reasons for the success of additive models are their increased flexibility when comp...
------------------------------------------------------------

2. 创建数据集...
3. 创建DataLoader...

✓ 数据准备完成
  训练样本数: 32094
  Batch大小: 8
  Batch数量: 4012
  每个epoch步数: 4012
  使用梯度累积: 4 步
  有效batch size: 32


## 8️⃣ 训练循环

In [9]:
print("="*60)
print("开始训练")
print("="*60)

# 设置优化器
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config.learning_rate,
    weight_decay=0.01
)

# 设置学习率调度器
total_steps = len(train_dataloader) * config.num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=config.warmup_steps,
    num_training_steps=total_steps
)

print(f"\n训练配置:")
print(f"  总步数: {total_steps}")
print(f"  预热步数: {config.warmup_steps}")
print(f"  保存间隔: {config.save_steps} 步")
print(f"  梯度裁剪: {config.max_grad_norm}")
print()

# 训练循环
model.train()
train_losses = []
global_step = 0
best_loss = float('inf')
start_time = time.time()

for epoch in range(config.num_epochs):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch + 1}/{config.num_epochs}")
    print(f"{'='*60}")
    
    epoch_losses = []
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
    
    for batch_idx, batch in enumerate(progress_bar):
        try:
            # 移动数据到设备
            input_ids = batch["input_ids"].to(config.device)
            attention_mask = batch["attention_mask"].to(config.device)
            
            # 采样时间步
            batch_size = input_ids.shape[0]
            t = torch.rand(batch_size, device=config.device)
            
            # 前向diffusion
            xt, mask = forward_diffusion_step(
                input_ids,
                t,
                config.mask_token_id,
                device=config.device
            )
            
            # 模型前向传播
            logits = model(xt, attention_mask=attention_mask)
            
            # 计算损失
            loss = compute_diffusion_loss(logits, input_ids, mask, t)
            
            # 跳过异常批次
            if loss is None:
                print(f"\n⚠️  步数 {global_step}: 跳过异常批次")
                continue
            
            # 梯度累积
            loss = loss / config.gradient_accumulation_steps
            loss.backward()
            
            # 每隔gradient_accumulation_steps更新一次
            if (batch_idx + 1) % config.gradient_accumulation_steps == 0:
                # 梯度裁剪
                torch.nn.utils.clip_grad_norm_(
                    model.parameters(),
                    config.max_grad_norm
                )
                
                # 更新参数
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                
                global_step += 1
            
            # 记录损失
            current_loss = loss.item() * config.gradient_accumulation_steps
            train_losses.append(current_loss)
            epoch_losses.append(current_loss)
            
            # 更新进度条
            progress_bar.set_postfix({
                'loss': f'{current_loss:.4f}',
                'lr': f'{scheduler.get_last_lr()[0]:.2e}'
            })
            
            # 保存checkpoint
            if global_step % config.save_steps == 0 and global_step > 0:
                checkpoint_path = os.path.join(
                    config.output_dir,
                    f"checkpoint-{global_step}"
                )
                os.makedirs(checkpoint_path, exist_ok=True)
                
                torch.save({
                    'epoch': epoch,
                    'global_step': global_step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'loss': current_loss,
                    'config': config
                }, os.path.join(checkpoint_path, 'pytorch_model.bin'))
                
                print(f"\n💾 Checkpoint保存: {checkpoint_path}")
                
                # 保存最佳模型
                avg_loss = sum(epoch_losses[-100:]) / min(100, len(epoch_losses))
                if avg_loss < best_loss:
                    best_loss = avg_loss
                    best_model_path = os.path.join(config.output_dir, "best_model")
                    os.makedirs(best_model_path, exist_ok=True)
                    torch.save({
                        'model_state_dict': model.state_dict(),
                        'loss': best_loss
                    }, os.path.join(best_model_path, 'pytorch_model.bin'))
                    print(f"🏆 最佳模型更新: loss={best_loss:.4f}")
        
        except Exception as e:
            print(f"\n❌ 批次 {batch_idx} 出错: {e}")
            continue
    
    # Epoch总结
    if epoch_losses:
        avg_epoch_loss = sum(epoch_losses) / len(epoch_losses)
        print(f"\nEpoch {epoch + 1} 完成")
        print(f"  平均Loss: {avg_epoch_loss:.4f}")
        print(f"  最佳Loss: {best_loss:.4f}")
        print(f"  完成步数: {global_step}")

# 训练结束
elapsed_time = time.time() - start_time
print("\n" + "="*60)
print("✅ 训练完成！")
print("="*60)
print(f"总训练时间: {elapsed_time/3600:.2f} 小时")
print(f"总步数: {global_step}")
print(f"最佳Loss: {best_loss:.4f}")
print(f"最终Loss: {train_losses[-1]:.4f}" if train_losses else "N/A")
print(f"模型保存位置: {config.output_dir}")
print("="*60)

开始训练

训练配置:
  总步数: 12036
  预热步数: 500
  保存间隔: 1000 步
  梯度裁剪: 0.5


Epoch 1/3


Epoch 1:   0%|          | 0/4012 [00:00<?, ?it/s]


💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-1000
🏆 最佳模型更新: loss=9.3457

💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-1000

💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-1000

💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-1000
🏆 最佳模型更新: loss=9.3385

Epoch 1 完成
  平均Loss: 9.7569
  最佳Loss: 9.3385
  完成步数: 1003

Epoch 2/3


Epoch 2:   0%|          | 0/4012 [00:00<?, ?it/s]


💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-2000
🏆 最佳模型更新: loss=8.9585

💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-2000
🏆 最佳模型更新: loss=8.9140

💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-2000
🏆 最佳模型更新: loss=8.9025

💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-2000
🏆 最佳模型更新: loss=8.8759

Epoch 2 完成
  平均Loss: 9.2432
  最佳Loss: 8.8759
  完成步数: 2006

Epoch 3/3


Epoch 3:   0%|          | 0/4012 [00:00<?, ?it/s]


💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-3000
🏆 最佳模型更新: loss=8.7752

💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-3000

💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-3000

💾 Checkpoint保存: /root/autodl-tmp/diffusion_galactica_checkpoints/checkpoint-3000

Epoch 3 完成
  平均Loss: 9.0166
  最佳Loss: 8.7752
  完成步数: 3009

✅ 训练完成！
总训练时间: 0.27 小时
总步数: 3009
最佳Loss: 8.7752
最终Loss: 8.6378
模型保存位置: /root/autodl-tmp/diffusion_galactica_checkpoints


## 9️⃣ 训练可视化

In [ ]:
# 绘制训练损失曲线
if len(train_losses) > 0:
    plt.figure(figsize=(14, 5))
    
    # 原始loss曲线
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, alpha=0.6, linewidth=0.8)
    plt.xlabel('训练步数', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.title('训练损失曲线', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # 平滑loss曲线
    plt.subplot(1, 2, 2)
    window = min(100, len(train_losses) // 10)
    if len(train_losses) > window:
        smoothed = np.convolve(train_losses, np.ones(window)/window, mode='valid')
        plt.plot(smoothed, linewidth=2, color='#2E86AB')
        plt.xlabel('训练步数', fontsize=12)
        plt.ylabel('平滑Loss', fontsize=12)
        plt.title(f'训练损失 (移动平均, window={window})', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # 保存图片
    loss_plot_path = os.path.join(config.output_dir, 'training_loss.png')
    plt.savefig(loss_plot_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n📊 训练曲线已保存: {loss_plot_path}")
    
    # 训练统计
    print("\n训练统计:")
    print(f"  最终Loss: {train_losses[-1]:.4f}")
    print(f"  最佳Loss: {best_loss:.4f}")
    if len(train_losses) > 1:
        improvement = (train_losses[0] - train_losses[-1]) / train_losses[0] * 100
        print(f"  Loss降低: {improvement:.1f}%")
    print(f"  总步数: {len(train_losses)}")
else:
    print("⚠️  没有训练损失数据可视化")

## 🔟 生成测试

In [ ]:
def sample_diffusion(
    model,
    tokenizer,
    prompt="",
    num_steps=64,
    max_length=128,
    temperature=0.9,
    device="cuda"
):
    """
    使用diffusion模型生成文本
    
    Args:
        model: 训练好的diffusion模型
        tokenizer: tokenizer
        prompt: 提示文本
        num_steps: 反向diffusion步数
        max_length: 最大生成长度
        temperature: 采样温度
        device: 设备
    """
    model.eval()
    mask_token_id = config.mask_token_id
    
    # 编码prompt
    if prompt:
        prompt_ids = tokenizer.encode(prompt, add_special_tokens=True)
        prompt_len = len(prompt_ids)
    else:
        prompt_ids = [tokenizer.bos_token_id] if tokenizer.bos_token_id else [0]
        prompt_len = 1
    
    # 初始化序列（全mask）
    total_len = min(max_length, 512)
    xt = torch.full((1, total_len), mask_token_id, dtype=torch.long, device=device)
    xt[0, :prompt_len] = torch.tensor(prompt_ids[:prompt_len], device=device)
    
    # 反向diffusion过程
    with torch.no_grad():
        for step in tqdm(reversed(range(1, num_steps + 1)), desc="生成中", leave=False):
            t_current = step / num_steps
            t_prev = (step - 1) / num_steps
            
            # 模型预测
            logits = model(xt)
            
            # 数值稳定性处理
            if torch.isnan(logits).any() or torch.isinf(logits).any():
                logits = torch.nan_to_num(logits, nan=0.0, posinf=1e4, neginf=-1e4)
            
            logits = logits / temperature
            
            # 计算概率并采样
            probs = F.softmax(logits, dim=-1)
            
            # 检查概率有效性
            if torch.isnan(probs).any() or torch.isinf(probs).any():
                vocab_size = probs.shape[-1]
                probs = torch.ones_like(probs) / vocab_size
            
            try:
                x0_pred = torch.multinomial(probs.view(-1, probs.shape[-1]), 1).view(1, -1)
            except:
                x0_pred = torch.argmax(probs, dim=-1)
            
            # 更新序列
            alpha_t = 1 - t_current
            alpha_s = 1 - t_prev
            
            is_masked = (xt[0] == mask_token_id)
            jump_prob = (alpha_s - alpha_t) / (1 - alpha_t + 1e-8) if step > 1 else 1.0
            
            should_jump = torch.rand(total_len, device=device) < jump_prob
            update_mask = is_masked & should_jump
            
            xt[0, update_mask] = x0_pred[0, update_mask]
            
            # 保持prompt不变
            xt[0, :prompt_len] = torch.tensor(prompt_ids[:prompt_len], device=device)
    
    # 解码
    generated_ids = xt[0].cpu().tolist()
    generated_ids = [tid for tid in generated_ids if tid != mask_token_id]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    return generated_text


print("✓ 生成函数定义完成")

In [ ]:
# 生成测试
print("\n" + "="*60)
print("🎯 生成测试")
print("="*60)

test_prompts = [
    "The theory of relativity",
    "Artificial intelligence",
    "Quantum mechanics",
    "Climate change",
    "Deep learning"
]

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n{i}. Prompt: '{prompt}'")
    print("-" * 50)
    
    try:
        generated = sample_diffusion(
            model,
            tokenizer,
            prompt=prompt,
            num_steps=64,
            max_length=128,
            temperature=0.9,
            device=config.device
        )
        print(f"生成: {generated}")
    except Exception as e:
        print(f"生成失败: {e}")

print("\n" + "="*60)
print("✅ 生成测试完成")
print("="*60)

## 📝 使用说明

### 数据准备

在运行此notebook之前，请确保:

1. **下载ArXiv数据集**
   - 从Hugging Face下载arxiv-summarization数据集的parquet文件

2. **处理数据**
   ```bash
   python process_arxiv_dataset.py /path/to/parquet/files
   ```

3. **确认数据文件存在**
   - 确保 `./processed_data/arxiv_train.txt` 文件已生成

### 训练配置

**快速测试** (在第2节修改config):
```python
config.max_train_samples = 1000
config.batch_size = 4
config.max_length = 64
config.num_epochs = 1
```

**正式训练**:
```python
config.max_train_samples = None  # 使用全部数据
config.batch_size = 8
config.max_length = 128
config.num_epochs = 3
```

### 训练监控

- 训练过程中会自动保存checkpoint
- 最佳模型保存在 `{output_dir}/best_model/`
- 训练曲线保存在 `{output_dir}/training_loss.png`

### 常见问题

1. **内存不足**: 减小batch_size或max_length
2. **NaN/Inf错误**: 已有数值稳定性处理，会自动跳过异常批次
3. **训练太慢**: 可以先用小数据集测试（设置max_train_samples）

---

**祝训练顺利！** 🎉